# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 27 2022 2:44PM,242891,10,8330555,Bio-PRF,Released
1,Jul 27 2022 2:40PM,242886,20,ALU0007373,Alumier Labs Inc.,Released
2,Jul 27 2022 2:40PM,242886,20,ALUR002388355,Alumier Labs Inc.,Released
3,Jul 27 2022 2:40PM,242886,20,ALUR017061265,Alumier Labs Inc.,Released
4,Jul 27 2022 2:40PM,242886,20,ALUR084902364,Alumier Labs Inc.,Released
5,Jul 27 2022 2:40PM,242886,20,ALUR144237517,Alumier Labs Inc.,Released
6,Jul 27 2022 2:40PM,242886,20,ALUR253218582,Alumier Labs Inc.,Released
7,Jul 27 2022 2:40PM,242886,20,ALUR599304306,Alumier Labs Inc.,Released
8,Jul 27 2022 2:40PM,242886,20,ALUR924389896,Alumier Labs Inc.,Released
9,Jul 27 2022 2:40PM,242886,20,ALUR738194250,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,242867,Released,1
33,242877,Released,1
34,242886,Released,20
35,242887,Released,1
36,242891,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242867,NaN,NaN,1.0
242877,NaN,NaN,1.0
242886,NaN,NaN,20.0
242887,NaN,NaN,1.0
242891,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242798,0.0,0.0,2.0
242806,0.0,0.0,1.0
242808,17.0,3.0,4.0
242810,0.0,0.0,8.0
242818,14.0,6.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242798,0,0,2
242806,0,0,1
242808,17,3,4
242810,0,0,8
242818,14,6,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242798,0,0,2
1,242806,0,0,1
2,242808,17,3,4
3,242810,0,0,8
4,242818,14,6,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242798,,,2
1,242806,,,1
2,242808,17,3,4
3,242810,,,8
4,242818,14,6,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 27 2022 2:44PM,242891,10,Bio-PRF
1,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.
21,Jul 27 2022 2:16PM,242887,12,LF of America Corp.
22,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America
23,Jul 27 2022 12:57PM,242867,10,Yusen – 3D Matrix
24,Jul 27 2022 12:50PM,242866,10,Yusen – 3D Matrix
25,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc."
33,Jul 27 2022 12:10PM,242861,10,Bio-PRF
52,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.
56,Jul 27 2022 11:48AM,242858,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 27 2022 2:44PM,242891,10,Bio-PRF,,,1
1,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.,,,20
2,Jul 27 2022 2:16PM,242887,12,LF of America Corp.,,,1
3,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America,,,1
4,Jul 27 2022 12:57PM,242867,10,Yusen – 3D Matrix,,,1
5,Jul 27 2022 12:50PM,242866,10,Yusen – 3D Matrix,,,1
6,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc.",,,8
7,Jul 27 2022 12:10PM,242861,10,Bio-PRF,,11,8
8,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.,,1,3
9,Jul 27 2022 11:48AM,242858,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 2:44PM,242891,10,Bio-PRF,1,,
1,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.,20,,
2,Jul 27 2022 2:16PM,242887,12,LF of America Corp.,1,,
3,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America,1,,
4,Jul 27 2022 12:57PM,242867,10,Yusen – 3D Matrix,1,,
5,Jul 27 2022 12:50PM,242866,10,Yusen – 3D Matrix,1,,
6,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc.",8,,
7,Jul 27 2022 12:10PM,242861,10,Bio-PRF,8,11,
8,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.,3,1,
9,Jul 27 2022 11:48AM,242858,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 2:44PM,242891,10,Bio-PRF,1,,
1,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.,20,,
2,Jul 27 2022 2:16PM,242887,12,LF of America Corp.,1,,
3,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America,1,,
4,Jul 27 2022 12:57PM,242867,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 2:44PM,242891,10,Bio-PRF,1,NaN,NaN
1,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.,20,NaN,NaN
2,Jul 27 2022 2:16PM,242887,12,LF of America Corp.,1,NaN,NaN
3,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America,1,NaN,NaN
4,Jul 27 2022 12:57PM,242867,10,Yusen – 3D Matrix,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 2:44PM,242891,10,Bio-PRF,1,0.0,0.0
1,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.,20,0.0,0.0
2,Jul 27 2022 2:16PM,242887,12,LF of America Corp.,1,0.0,0.0
3,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America,1,0.0,0.0
4,Jul 27 2022 12:57PM,242867,10,Yusen – 3D Matrix,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2671361,42,27.0,1.0
12,242887,1,0.0,0.0
15,242808,4,3.0,17.0
16,242877,1,0.0,0.0
19,971253,5,6.0,14.0
20,728567,51,1.0,4.0
21,971426,4,0.0,0.0
22,485703,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2671361,42,27.0,1.0
1,12,242887,1,0.0,0.0
2,15,242808,4,3.0,17.0
3,16,242877,1,0.0,0.0
4,19,971253,5,6.0,14.0
5,20,728567,51,1.0,4.0
6,21,971426,4,0.0,0.0
7,22,485703,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42,27.0,1.0
1,12,1,0.0,0.0
2,15,4,3.0,17.0
3,16,1,0.0,0.0
4,19,5,6.0,14.0
5,20,51,1.0,4.0
6,21,4,0.0,0.0
7,22,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,1.0
2,15,Released,4.0
3,16,Released,1.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,17.0,0.0,14.0,4.0,0.0,0.0
Executing,27.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0
Released,42.0,1.0,4.0,1.0,5.0,51.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,17.0,0.0,14.0,4.0,0.0,0.0
1,Executing,27.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0
2,Released,42.0,1.0,4.0,1.0,5.0,51.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,17.0,0.0,14.0,4.0,0.0,0.0
1,Executing,27.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0
2,Released,42.0,1.0,4.0,1.0,5.0,51.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()